In [1]:
import json
import os
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
import sklearn.utils

In [2]:
df_list = []
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/res_091110") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)

In [3]:
df_columns_list = ["udid","gender"]
for i in range(10):
    df_columns_list.append("c{}".format(i))
df.columns = df_columns_list

In [4]:
df

,udid,gender,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,1,男,377218,379528,380043,380126,379533,379304,379537,379451,379489,378183
1,100326,女,375430,380316,381443,380482,375411,380587,379338,375166,380486,375263
2,10113,男,378036,377873,378020,378051,372948,378047,378033,378011,378025,373500
3,10140,男,381262,381295,381284,381316,380955,381270,381253,381287,381309,381349
4,101966,男,381295,381239,380393,381284,380400,381316,381287,381309,378462,380381
...,...,...,...,...,...,...,...,...,...,...,...,...
5668,88973,男,131762,239963,380951,238339,380920,380926,33352,380841,44591,380984
5669,89756,男,378162,364501,380043,376909,380316,374925,374021,377014,375524,380794
5670,91466,男,380549,380542,373479,379568,377928,380600,378982,380482,380550,380587
5671,92106,女,379630,380768,379528,380669,371589,379724,380482,379607,379899,380196


In [5]:
a_dict = {}
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/title_tokenize.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        a_dict[aid] = {}
        title_tokenize = temp_dict["title_tokenize"]
        a_dict[aid]["title_tokenize"] = title_tokenize
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/content_tokenize.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        if not aid in a_dict:
            print(aid)
            continue
        content_tokenize = temp_dict["content_tokenize"][:512]
        a_dict[aid]["content_tokenize"] = content_tokenize

In [7]:
text_list = []
y_list = []
for index in df.index:
    y = df.loc[index,"gender"]
    temp_text_list = []
    for i in range(10):
        c_text_list = []
        c = "c{}".format(i)
        aid = str(df.loc[index,c]).strip()
        if aid:
            if aid in a_dict:
                c_text_list = [*a_dict[aid]["title_tokenize"],"<PADDING>",*a_dict[aid]["content_tokenize"][:512]]
            else:
                print("aid不存在",index,c,aid)
        else:
            print(index)
        temp_text_list.append(c_text_list)     
    text_list.append(temp_text_list)
    y_list.append(y)

20
50
aid不存在 57 c7 0
107
122
aid不存在 122 c8 130
158
178
184
189
aid不存在 204 c4 0
210
214
235
248
250
289
296
304
314
322
347
382
388
480
483
aid不存在 483 c9 130
aid不存在 514 c9 0
621
aid不存在 633 c2 213
aid不存在 643 c3 0
651
aid不存在 674 c6 0
688
aid不存在 726 c7 0
804
861
868
aid不存在 917 c9 0
938
944
1002
aid不存在 1004 c9 0
1125
1175
1220
aid不存在 1223 c7 0
aid不存在 1319 c9 0
1398
aid不存在 1464 c9 0
aid不存在 1466 c9 0
1479
aid不存在 1492 c5 0
1599
aid不存在 1599 c9 0
1687
aid不存在 1726 c9 0
1855
1891
1939
2003
2033
2059
2065
aid不存在 2150 c9 0
2154
2162
aid不存在 2211 c9 0
aid不存在 2212 c7 0
2394
aid不存在 2418 c9 0
aid不存在 2421 c4 0
2427
2465
2584
aid不存在 2590 c5 0
2591
2593
2595
2620
2622
aid不存在 2774 c7 0
2836
2859
aid不存在 2915 c9 0
2959
aid不存在 2963 c5 0
2973
aid不存在 3169 c6 0
3196
3251
3262
aid不存在 3286 c5 134
aid不存在 3293 c8 0
3319
3380
aid不存在 3388 c7 0
aid不存在 3404 c9 0
3410
3430
3494
aid不存在 3506 c8 0
aid不存在 3534 c6 0
aid不存在 3573 c9 0
3623
aid不存在 3731 c9 0
3790
3835
3837
aid不存在 3841 c3 0
aid不存在 3854 c8 0
aid不存在 3862 c9 0
3869
387

In [8]:
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/gender/最终数据集tokenize.jsonl","w") as f:
    for tag,content_tokenize in zip(y_list,text_list):
        f.write(json.dumps({"tag":tag,"all_content_tokenize":content_tokenize},ensure_ascii=False) + "\n")

In [1]:
import json
import os
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
import sklearn.utils

class Gender_data_deal(object):
    
    def gender_data_output(self,data_input_path,title_tokenize_path,content_tokenize_path,position_output_data_path):
        df_list = []
        with open(data_input_path) as f:
            for line in f:
                df_list.append(line.strip().split('\001'))
        df = pd.DataFrame(df_list)
        df_columns_list = ["udid","gender"]
        for i in range(10):
            df_columns_list.append("c{}".format(i))
        df.columns = df_columns_list
        a_dict = {}
        with open(title_tokenize_path) as f:
            for line in f:
                temp_dict = json.loads(line.strip())
                aid = temp_dict["aid"]
                a_dict[aid] = {}
                title_tokenize = temp_dict["title_tokenize"]
                a_dict[aid]["title_tokenize"] = title_tokenize
        with open(content_tokenize_path) as f:
            for line in f:
                temp_dict = json.loads(line.strip())
                aid = temp_dict["aid"]
                if not aid in a_dict:
                    print(aid)
                    continue
                content_tokenize = temp_dict["content_tokenize"][:512]
                a_dict[aid]["content_tokenize"] = content_tokenize
        text_list = []
        y_list = []
        for index in df.index:
            y = df.loc[index,"gender"]
            temp_text_list = []
            for i in range(10):
                c_text_list = []
                c = "c{}".format(i)
                aid = str(df.loc[index,c]).strip()
                if aid:
                    if aid in a_dict:
                        c_text_list = [*a_dict[aid]["title_tokenize"],"<PADDING>",*a_dict[aid]["content_tokenize"][:512]]
                    else:
                        print("aid不存在",index,c,aid)
                else:
                    print(index)
                temp_text_list.append(c_text_list)     
            text_list.append(temp_text_list)
            y_list.append(y)
        with open(gender_output_data_path,"w") as f:
            for tag,content_tokenize in zip(y_list,text_list):
                f.write(json.dumps({"tag":tag,"all_content_tokenize":content_tokenize},ensure_ascii=False) + "\n")

In [2]:
data_input_path = "/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/res_091110"
title_tokenize_path = "/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/title_tokenize.jsonl"
content_tokenize_path = "/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/content_tokenize.jsonl"
gender_output_data_path = "/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/gender/最终数据集tokenize.jsonl"
gender = Gender_data_deal()
gender.gender_data_output(data_input_path,title_tokenize_path,content_tokenize_path,gender_output_data_path)

20
50
aid不存在 57 c7 0
107
122
aid不存在 122 c8 130
158
178
184
189
aid不存在 204 c4 0
210
214
235
248
250
289
296
304
314
322
347
382
388
480
483
aid不存在 483 c9 130
aid不存在 514 c9 0
621
aid不存在 633 c2 213
aid不存在 643 c3 0
651
aid不存在 674 c6 0
688
aid不存在 726 c7 0
804
861
868
aid不存在 917 c9 0
938
944
1002
aid不存在 1004 c9 0
1125
1175
1220
aid不存在 1223 c7 0
aid不存在 1319 c9 0
1398
aid不存在 1464 c9 0
aid不存在 1466 c9 0
1479
aid不存在 1492 c5 0
1599
aid不存在 1599 c9 0
1687
aid不存在 1726 c9 0
1855
1891
1939
2003
2033
2059
2065
aid不存在 2150 c9 0
2154
2162
aid不存在 2211 c9 0
aid不存在 2212 c7 0
2394
aid不存在 2418 c9 0
aid不存在 2421 c4 0
2427
2465
2584
aid不存在 2590 c5 0
2591
2593
2595
2620
2622
aid不存在 2774 c7 0
2836
2859
aid不存在 2915 c9 0
2959
aid不存在 2963 c5 0
2973
aid不存在 3169 c6 0
3196
3251
3262
aid不存在 3286 c5 134
aid不存在 3293 c8 0
3319
3380
aid不存在 3388 c7 0
aid不存在 3404 c9 0
3410
3430
3494
aid不存在 3506 c8 0
aid不存在 3534 c6 0
aid不存在 3573 c9 0
3623
aid不存在 3731 c9 0
3790
3835
3837
aid不存在 3841 c3 0
aid不存在 3854 c8 0
aid不存在 3862 c9 0
3869
387

In [5]:
df.isna().sum()

udid      0
gender    0
c0        0
c1        0
c2        0
c3        0
c4        0
c5        0
c6        0
c7        0
c8        0
c9        0
dtype: int64

In [6]:
collections.Counter(df["gender"])

Counter({'男': 4579, '女': 1094})

In [8]:
a_list = []
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/aid_title") as f:
    for line in f:
        a_list.append(line.strip().split('\001'))

In [9]:
#a_df = pd.read_csv("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/aid_title",sep='\001',header=None)
a_df = pd.DataFrame(a_list)

In [10]:
a_df.columns = ["aid","title"]

a_df

,aid,title
0,27,来自前Google产品经理的建议111
1,64,【HOW】大众点评能够成功的三个关键操作点
2,69,谷歌不得已用白色？机缘巧合成个性
3,75,下对6步棋，微软重获魔力
4,92,雅虎CEO:要新生，雅虎必须裁员
...,...,...
324638,381406,欧洲央行或将有意识控制欧元升值
324639,381530,纽约三大股指反抽，创业板低价股遇打击，明天能否见底？
324640,381535,脱口秀大会还能走多远，关键看怎么处理这六大关系
324641,381550,微信低调上线新功能，将影响未来的使用习惯？


In [11]:
a_df.isna().sum()

aid      0
title    0
dtype: int64

In [12]:
a_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324643 entries, 0 to 324642
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   aid     324643 non-null  object
 1   title   324643 non-null  object
dtypes: object(2)
memory usage: 5.0+ MB


In [13]:
a_dict = {}
for index in a_df.index:
    a_dict[a_df.loc[index,"aid"]] = a_df.loc[index,"title"]

In [14]:
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/title.jsonl","w") as f:
    for index in a_df.index:
        aid = a_df.loc[index,"aid"]
        title = a_df.loc[index,"title"]
        f.write(json.dumps({"aid":str(aid),"title":title.lower()},ensure_ascii=False) + "\n")

In [15]:
#del a_dict
a_dict = {}
with open("/mnt/d/zourui/predict_user_attribute20201214/user_attribute20201231/user_profile/title_res.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        title_tokenize = temp_dict["title_tokenize"]
        a_dict[aid] = title_tokenize

In [16]:
a_dict['377218']

['利率', '红线', '降低', '后', '，', '民间', '借贷', '更', '难', '了']

In [17]:
df.iloc[20]

udid       11972
gender         男
c0        379344
c1        380126
c2        373913
c3        374021
c4        346193
c5        374412
c6        379168
c7        379187
c8              
c9        374616
Name: 20, dtype: object

In [18]:
a_dict["27"]

['来自', '前', 'google', '产品', '经理', '的', '建议']

In [19]:
title_tokenize_list = []
gender_list = []
for index in df.index:
    gender = df.loc[index,"gender"]
    temp_list = []
    for i in range(10):
        c = "c{}".format(i)
        if str(df.loc[index,c]):
            if str(df.loc[index,c]) in a_dict:
                temp_list.extend(a_dict[str(df.loc[index,c])])
            else:
                print(index,c,df.loc[index,c])
        else:
            print(index)
    gender_list.append(gender)
    title_tokenize_list.append(temp_list)

20
50
57 c7 0
107
122
122 c8 130
158
178
184
189
204 c4 0
210
214
235
248
250
289
296
304
314
322
347
382
388
480
483
483 c9 130
514 c9 0
621
633 c2 213
643 c3 0
651
674 c6 0
688
726 c7 0
804
861
868
917 c9 0
938
944
1002
1004 c9 0
1125
1175
1220
1223 c7 0
1319 c9 0
1398
1464 c9 0
1466 c9 0
1479
1492 c5 0
1599
1599 c9 0
1687
1726 c9 0
1855
1891
1939
2003
2033
2059
2065
2150 c9 0
2154
2162
2211 c9 0
2212 c7 0
2394
2418 c9 0
2421 c4 0
2427
2465
2584
2590 c5 0
2591
2593
2595
2620
2622
2774 c7 0
2836
2859
2915 c9 0
2959
2963 c5 0
2973
3169 c6 0
3196
3251
3262
3286 c5 134
3293 c8 0
3319
3380
3388 c7 0
3404 c9 0
3410
3430
3494
3506 c8 0
3534 c6 0
3573 c9 0
3623
3731 c9 0
3790
3835
3837
3841 c3 0
3854 c8 0
3862 c9 0
3869
3871
3913
3939
4000
4010 c6 0
4012
4022
4032 c8 0
4041 c0 110
4054 c2 0
4171 c7 0
4173
4199
4216
4228 c5 0
4254 c4 0
4305
4370
4441
4443
4476
4525 c4 0
4538
4563
4566 c7 0
4609
4651 c9 0
4698 c9 0
4722 c2 0
4734 c9 0
4740
4891 c4 0
4897
4940 c7 0
4941
4955 c9 0
4977 c3 0
4978

In [20]:
output_dict = {}

In [21]:
with open("/home/zourui/data/predict_user_attribute20200911/raw_data/gender/data/最终数据集tokenize.jsonl","w") as f:
    for tag,content_tokenize in zip(gender_list,title_tokenize_list):
        if tag == "女":
            for _ in range(4):
                f.write(json.dumps({"tag":tag,"content_tokenize":content_tokenize},ensure_ascii=False) + "\n")
        else:
            f.write(json.dumps({"tag":tag,"content_tokenize":content_tokenize},ensure_ascii=False) + "\n")
            
    

In [86]:
sklearn.utils.shuffle([[1],[2],[3]],[1,2,3])

[[[1], [2], [3]], [1, 2, 3]]

In [3]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
with strategy.scope():
    model = tf.keras.models.load_model("/home/zhaodachuan/jupyter_lab/user_profile/gender/model_output")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2020-09-11 18:26:15,476-INFO: Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [4]:
a_dict = {}
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/title_res.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        title_tokenize = temp_dict["title_tokenize"]
        a_dict[aid] = title_tokenize

In [5]:
a_dict["27"]

['来自', '前', 'google', '产品', '经理', '的', '建议']

In [6]:
predict_list = []
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/pre_sex_all_091116") as f:
    for line in f:
        predict_list.append(line.strip().split('\001'))
predict_df = pd.DataFrame(predict_list)

In [7]:
predict_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,男,380021,379528,380043,380126,379521,379529,379451,379526,378915,379489
1,6,女,381448,381295,381385,381284,381359,381552,381393,381540,381443,381309
2,8,男,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
3,38,男,366344,366247,366434,366043,366221,366113,366195,366379,366163,366182
4,44,,378980,378933,379975,380021,377002,380043,376563,380126,378967,376989
...,...,...,...,...,...,...,...,...,...,...,...,...
261362,2902765,,381483,381543,381474,381547,381539,381559,381552,381508,381542,381523
261363,2902766,,381437,381474,381547,381313,362843,381542,381508,379878,380998,381523
261364,2902770,,381453,381543,381474,381547,381537,381542,381535,381510,381540,381541
261365,2902775,,381483,381385,381439,381559,381537,381552,381508,381510,381540,381523


In [8]:
df_columns_list = ["udid","gender"]
for i in range(10):
    df_columns_list.append("c{}".format(i))
predict_df.columns = df_columns_list

In [9]:
predict_df

,udid,gender,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,1,男,380021,379528,380043,380126,379521,379529,379451,379526,378915,379489
1,6,女,381448,381295,381385,381284,381359,381552,381393,381540,381443,381309
2,8,男,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
3,38,男,366344,366247,366434,366043,366221,366113,366195,366379,366163,366182
4,44,,378980,378933,379975,380021,377002,380043,376563,380126,378967,376989
...,...,...,...,...,...,...,...,...,...,...,...,...
261362,2902765,,381483,381543,381474,381547,381539,381559,381552,381508,381542,381523
261363,2902766,,381437,381474,381547,381313,362843,381542,381508,379878,380998,381523
261364,2902770,,381453,381543,381474,381547,381537,381542,381535,381510,381540,381541
261365,2902775,,381483,381385,381439,381559,381537,381552,381508,381510,381540,381523


In [10]:
title_tokenize_list = []
gender_list = []
udid_list = []
df = predict_df
for index in df.index:
    udid = df.loc[index,"udid"]
    udid_list.append(udid)
    gender = df.loc[index,"gender"]
    gender_list.append(gender)
    temp_list = []
    for i in range(10):
        c = "c{}".format(i)
        if str(df.loc[index,c]):
            if str(df.loc[index,c]) in a_dict:
                temp_list.extend(a_dict[str(df.loc[index,c])])
            else:
                print(index,c,df.loc[index,c])
        else:
            print(index)
    title_tokenize_list.append(temp_list)

45 c8 134
62 c3 130
75 c3 130
351 c8 134
424 c7 134
429 c5 130
609 c3 130
764 c4 130
881 c3 130
1233 c4 130
1544 c5 161
1695 c4 154
2131 c3 130
2163 c3 130
2818 c8 96
2859 c5 130
2877 c3 130
3096 c7 130
3492 c6 130
3730 c8 96
4812 c8 161
5074 c5 161
5525 c3 130
5629 c3 130
5808 c3 130
6014 c8 142
6779 c9 96
6834 c2 130
7343 c4 130
7383 c2 130
7416 c4 130
7467 c2 161
7877 c0 108
8664 c4 130
8758 c2 135
8758 c3 161
8848 c6 130
8853 c6 134
8900 c5 130
9026 c7 126
9260 c8 134
9509 c5 213
9606 c4 130
9649 c5 161
9835 c6 96
10461 c0 169
10478 c6 154
11371 c4 130
11847 c5 130
12263 c4 161
12442 c3 130
13693 c4 130
14043 c4 130
14063 c6 161
14459 c6 134
15459 c0 156
15459 c3 154
15605 c4 130
15742 c4 161
16693 c1 108
17034 c7 161
17254 c8 155
17269 c3 130
17303 c1 130
17405 c2 130
17449 c0 130
18272 c1 108
18343 c7 126
18559 c5 130
21687 c5 130
22247 c5 161
22813 c9 134
22975 c8 134
26008 c8 134
26621 c1 130
26967 c3 122
28065 c5 65
29463 c2 130
29799 c4 130
31617 c8 126
32048 c3 130
32578 c3 

In [11]:
title_tokenize_list[0][:10]

['背叛', '乔布斯', '，', '库克', '做', '对', '了', '性能', '翻倍', '，']

In [12]:
APP_DIR = r"/home/zhaodachuan/jupyter_lab/user_profile/gender"
with open(os.path.join(APP_DIR,"data","word2index.json")) as f:
    word2index_dict = json.load(f)

In [13]:
def trans2index(word2index_dict,word):
    """转换"""
    if word in word2index_dict:
        return word2index_dict[word]
    else:
        if "<UNK>" in word2index_dict:
            return word2index_dict["<UNK>"]
        else:
            raise ValueError("没有这个值，请检查")

In [14]:
word2index_dict["我"]

378

In [15]:
title_emebdding_list = []
for tokenize_list in title_tokenize_list:
    title_emebdding_list.append([trans2index(word2index_dict,word) for word in tokenize_list])

In [16]:
title_emebdding_list[0]

[64145,
 60511,
 11,
 32049,
 1674,
 6,
 129,
 4495,
 11643,
 11,
 17079,
 2196,
 865,
 2754,
 30014,
 1993,
 73706,
 6870,
 1046,
 3289,
 325,
 48001,
 3560,
 8,
 4467,
 4216,
 1,
 129,
 325,
 17764,
 17760,
 819,
 31245,
 325,
 15708,
 17987,
 2827,
 10596,
 241,
 2011,
 13888,
 13888,
 129,
 6929,
 325,
 2703,
 1006,
 650,
 8626,
 483,
 27797,
 495,
 23545,
 13768,
 23546,
 58307,
 8,
 1,
 11,
 1467,
 2720,
 33473,
 8,
 1,
 3415,
 5602,
 647,
 16892,
 11,
 341,
 40,
 1882,
 129,
 325]

In [17]:
title_emebdding_npa = np.array(title_emebdding_list)

In [18]:
title_emebdding_npa = tf.keras.preprocessing.sequence.pad_sequences(title_emebdding_npa,maxlen=128,dtype=np.int32,truncating="post", padding='post',value=0)

In [19]:
title_emebdding_npa.shape

(261367, 128)

In [20]:
res_list = []
for i,chunk in enumerate(np.array_split(title_emebdding_npa, 1000)):
    #print(chunk)
    res = model.predict(chunk)
    for v in res:
        res_list.append(v[0])
    if i % 100 == 0:
        print(i)
    #break

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


2020-09-11 18:28:23,117-WARNING: From /home/zhaodachuan/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


0
100
200
300
400
500
600
700
800
900


In [21]:
res_npa = np.array(res_list,dtype=np.float32)

In [49]:
(res_npa < 0.5).sum() / len(res_npa)

0.7331912597994391

In [23]:
1 - (res_npa < 0.5).sum() / len(res_npa)

0.2668087402005609

In [28]:
x_npa = np.array([len(v) for v in title_tokenize_list])

In [43]:
np.percentile(x_npa,20)

88.0

In [40]:
x_npa

array([ 74, 115,  94, ..., 101,  96,  97])

14.545454545454545

In [48]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,protocol='binary')
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,(udid,res) in enumerate(zip(udid_list,res_list)):
            if res < 0.5:
                res = "男"
            else:
                res = "女"
            batch.put(udid.encode(), {"attribute:predict_gender".encode(): res.encode()})
            #print(udid.encode(), {"attribute:predict_gender".encode(): res.encode()})
            #break
            if i % 100000 == 0:
                print(i)

0
100000
200000
